In [1]:
BASE_PATH = '..\\data\\'
ORIGINAL_BASE_PATH = '..\\data\\original\\'
DIR_SEPARATOR = '\\' # /

LABEL_COLUMN = 'ICD10'
TEXT_COLUMN = 'Text'
PRED_CLASS = 'pred_class'
ORIGINAL_TEXT = 'original_text'
ALT_LABELS = 'alt_labels'

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np

In [3]:
import os
from os import path

In [4]:
file_name = '{0}train-4.csv'.format(BASE_PATH)
train_df = pd.read_csv(file_name)

In [5]:
file_name = '{0}test-4.csv'.format(BASE_PATH)
test_df = pd.read_csv(file_name)

In [6]:
file_name = '{0}dev-4.csv'.format(BASE_PATH)
dev_df = pd.read_csv(file_name)

In [7]:
train_df[TEXT_COLUMN] = train_df[TEXT_COLUMN].apply(lambda x:  str(x))
dev_df[TEXT_COLUMN] = dev_df[TEXT_COLUMN].apply(lambda x:  str(x))
test_df[TEXT_COLUMN] = test_df[TEXT_COLUMN].apply(lambda x:  str(x))

In [8]:
train_array = train_df[TEXT_COLUMN].to_numpy()
test_array = test_df[TEXT_COLUMN].to_numpy()
dev_array = dev_df[TEXT_COLUMN].to_numpy()

In [9]:
intersection_test = np.intersect1d(test_array, train_array)

In [10]:
intersection_dev = np.intersect1d(dev_array, train_array)

In [11]:
len(intersection_test)/len(test_array)

0.727554268345138

In [12]:
len(intersection_dev)/len(dev_array)

0.7280442590661569

In [13]:
test_df.head()

,Text,pred_class
0,оклузи закри цервикал кана виж стриктур матк шийк,4153
1,плазмодиум ебв,414
2,x примариа дискинезиа сакралис латералис,4826
3,корози разяждан нараняван каустич химич външ в...,5655
4,спастикус параплегиа ксксксв аутозома рецессив...,1952


In [14]:
!pip install nlpaug numpy matplotlib python-dotenv setuptools requests

In [15]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as nacw

In [16]:
# map bg to en keyboard in order to use the nlp aug keyboard augmenter
def keyboard_mapping(word, direction, start_positions = []):
    bg_en_map = { 'а':'a', 'б':'b', 'в':'w', 'г':'g', 'д':'d', 'е':'e', 'ж':'v', 
                 'з':'z', 'и':'i', 'й':'j', 'к':'k', 'л':'l', 'м':'m', 'н':'n', 'о':'o', 'п':'p', 'р':'r',
                 'с':'s', 'т':'t', 'у':'u', 'ф':'f', 'х':'h', 'ц':'c', 'ч':'`', 'ш':'[', 'щ':']', 'ъ':'y',
                 'ь':'x', 'ю':'\\', 'я':'q',
                 'А':'A', 'Б':'B', 'В':'W', 'Г':'G', 'Д':'D', 'Е':'E', 'Ж':'V', 
                 'З':'Z', 'И':'I', 'Й':'J', 'К':'K', 'Л':'L', 'М':'M', 'Н':'N', 'О':'O', 'П':'P', 'Р':'R',
                 'С':'S', 'Т':'T', 'Ъ':'U', 'Ф':'F', 'Х':'H', 'Ц':'C', 'Ч':'~', 'Ш':'{', 'Щ':'}', 'Ъ':'Y',
                 'ѝ':'X', 'Ю':'|', 'Я':'Q'}
    
    en_bg_map = {v: k for k, v in bg_en_map.items()}
    result = ''
    
    current_map = bg_en_map if direction == 1 else en_bg_map    
    positions = []
    
    i = 0;
    for character in word:        
        # handle non-mapped characters
        if (character in current_map) and ((len(start_positions) == 0) or (i in start_positions)):
            result += current_map[character]
            positions.append(i)
        else:
            result += character
        i += 1
        
    return result, positions

In [17]:
import random

def augment_text(text):
  aug_char = [
      nac.RandomCharAug(action="swap", aug_char_min=1, min_char=1),
      nac.RandomCharAug(action="delete", aug_char_min=1, min_char=1),
      nac.KeyboardAug(aug_char_max=1, min_char=1, include_special_char=False, include_numeric=False)
  ]

  aug_word = [ 
      #nacw.RandomWordAug(action="swap"),
      nac.RandomCharAug(action="swap", aug_char_min=1, min_char=1)
  ]
  tokens = text.split(' ')
  tokens_len = len(tokens)
  
  augmented_text = text
    
  if len(text) == 1:
    augmented_text = text
  elif tokens_len == 1:
      aug_number = random.randint(0, 2)
      if aug_number == 2: #keyboard        
        transliterated, positions = keyboard_mapping(text, 1)
        augmented_transliterated = aug_char[aug_number].augment(transliterated)
        augmented_text, end_positions = keyboard_mapping(augmented_transliterated, 2, positions)
      else:
        augmented_text = aug_char[aug_number].augment(text)
  else:
    aug_number = 0  #random.randrange(2)
    if aug_number == 0:
      random_word_number = random.randint(0, tokens_len-1)
      random_word = tokens[random_word_number]
        
      i = 1
      while len(random_word) < 2 and i < 10:
        random_word_number = random.randint(0, tokens_len-1)
        random_word = tokens[random_word_number]
        i += 1
        
      if len(random_word) >= 2:
          try: 
              augmented_word = aug_word[aug_number].augment(random_word)
              tokens[random_word_number] = augmented_word
              augmented_text = ' '.join(tokens)
          except:
            print(text)
      else:
        try:
            augmented_text = aug_word[0].augment(text)
        except:
            print(text)
    else:    
      try:  
          augmented_text = aug_word[aug_number].augment(text)
      except:
        print(text)

  if augmented_text == text:
    augmented_text = aug_char[1].augment(augmented_text)

  return augmented_text

In [18]:
augment_text('таз')

'атз'

In [19]:
new_test_df = pd.DataFrame({
        TEXT_COLUMN:test_df[TEXT_COLUMN].apply(lambda x: augment_text(x) if x in intersection_test else x),
        PRED_CLASS:test_df[PRED_CLASS],
        ORIGINAL_TEXT:test_df[TEXT_COLUMN]
        })

In [20]:
new_test_array = new_test_df[TEXT_COLUMN].to_numpy()

In [21]:
intersection_new_test = np.intersect1d(new_test_array, train_array)

In [22]:
len(intersection_new_test)/len(new_test_array)

0.0023227138473890974

In [24]:
new_dev_df = pd.DataFrame({
        TEXT_COLUMN:dev_df[TEXT_COLUMN].apply(lambda x: augment_text(x) if x in intersection_dev else x),
        PRED_CLASS:dev_df[PRED_CLASS],
        ORIGINAL_TEXT:dev_df[TEXT_COLUMN]
        })

c a e


In [25]:
new_dev_array = new_dev_df[TEXT_COLUMN].to_numpy()

In [26]:
intersection_new_dev = np.intersect1d(new_dev_array, train_array)

In [27]:
len(intersection_new_dev)/len(new_dev_array)

0.00202757502027575

In [29]:
df_train_dev = pd.concat([train_df, dev_df])
label_groups = df_train_dev.groupby(TEXT_COLUMN)
alt_labels_df = label_groups[PRED_CLASS].apply(lambda x: x.values).reset_index()
alt_labels_df.set_index(TEXT_COLUMN, inplace=True)

In [30]:
alt_labels_df.head()

,pred_class
Text,
0 genus cpam,"[4819, 4820]"
0 genus laryngotracheoesophageal vehementissimo imbri,"[4817, 4816]"
0 genus morbo glycogen repono,"[1594, 1593]"
0 reipublic s a carcinomate firmo nexu fulcit,"[1041, 1039]"
0 scaena pulmonis carcinoma,"[1051, 1048]"


In [31]:
label_groups_train = train_df.groupby(TEXT_COLUMN)
alt_labels_train_df = label_groups_train[PRED_CLASS].apply(lambda x: x.values).reset_index()
alt_labels_train_df.set_index(TEXT_COLUMN, inplace=True)

In [32]:
alt_labels_train_df.head()

,pred_class
Text,
0 genus cpam,[4819]
0 genus laryngotracheoesophageal vehementissimo imbri,"[4817, 4816]"
0 genus morbo glycogen repono,"[1594, 1593]"
0 reipublic s a carcinomate firmo nexu fulcit,"[1041, 1039]"
0 scaena pulmonis carcinoma,"[1051, 1048]"


In [33]:
def get_alt_labels(text, current_df, alt_df):
  if not(text in alt_df.index):  
    values = current_df[current_df[TEXT_COLUMN]==text][ORIGINAL_TEXT].values
    if len(values) > 0:
      text = values[0]
    else:
      print(text)

  if text in alt_df.index:
    labels = alt_df.at[text, PRED_CLASS]
    return labels
  return []

In [35]:
get_alt_labels('плазмодиум бев', new_test_df, alt_labels_df)

плазмодиум бев


[]

In [36]:
new_test_df[ALT_LABELS] = new_test_df[TEXT_COLUMN].apply(lambda x: get_alt_labels(x, new_test_df, alt_labels_df))

In [37]:
new_test_df.head()

,Text,pred_class,original_text,alt_labels
0,оклузи закри цервикал кана виж стриктур матк ишйк,4153,оклузи закри цервикал кана виж стриктур матк шийк,[4150]
1,плазмодиум евб,414,плазмодиум ебв,[413]
2,x примариа дискинезиа сакралис латрелаис,4826,x примариа дискинезиа сакралис латералис,[4825]
3,корози разяждан нараняван каустич химич външ в...,5655,корози разяждан нараняван каустич химич външ в...,[]
4,псастиксу параплегиа ксксксв аутозома рецессив...,1952,спастикус параплегиа ксксксв аутозома рецессив...,[1957]


In [40]:
new_test_df.to_csv('{0}test-4.csv'.format(BASE_PATH), header=True, index=False)

In [38]:
new_dev_df[ALT_LABELS] = new_dev_df[TEXT_COLUMN].apply(lambda x: get_alt_labels(x, new_dev_df, alt_labels_train_df))

In [39]:
new_dev_df.head()

,Text,pred_class,original_text,alt_labels
0,перверз певрезрен апетит,1808,перверз перверзен апетит,[1804]
1,стеноз цикатрициал белодроб артери ворден клап...,2568,стеноз цикатрициал белодроб артери вроден клап...,[2563]
2,сумам калвициес ареат,3470,сумма калвициес ареат,[3471]
3,хронич бъбреч недсотаътчонтс i ii ст,3952,хронич бъбреч недостатъчност i ii ст,[3954]
4,хик дика метафисеал кондродисплазиа,5063,хик дика метафисеал кондродисплазиа,[]


In [41]:
new_dev_df.to_csv('{0}dev-4.csv'.format(BASE_PATH), header=True, index=False)